<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# FIA Insects Model

_by Jeremy Freeman (CarbonPlan), October 26, 2020_

This notebook demos insect modelling.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.utils import resample
from carbonplan_forests import setup, plot, load, fit, utils

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
setup.plotting()

In [ ]:
%load_ext autoreload
%autoreload 2

Load the raw FIA grouped data

In [ ]:
df = load.fia(store='local', states='conus', group_repeats=True)
df = load.terraclim(
    store='local', 
    tlim=(int(df['year_0'].min()), 2020),
    data_vars=['ppt','tavg','pdsi','cwd','pet','vpd'], 
    data_aggs=['sum','mean','mean','sum','mean','mean'], 
    df=df, 
    group_repeats=True
)

In [ ]:
df = pd.read_csv('/Users/freeman/Dropbox (Personal)/fia_wide_with_terraclim_v4.csv', low_memory=False)

Compute derived variables

In [ ]:
df['mort_ratio'] = df['fraction_insect_1']#(df['mort_1'] / df['balive_0']) * df['disturb_insect_1'].astype('int')
df['age_squared'] = df['age'] ** 2
df['duration'] = (df['year_1'] - df['year_0'])

Set thresholds

In [ ]:
inds = (
    (df['condprop'] > 0.3) & 
    (df['disturb_human_1'] != True) & 
    (df['disturb_fire_1'] != True) & 
    (df['treatment_cutting_1'] != True)
)
df = df[inds]

Extract variables

In [ ]:
y = df['mort_ratio']
x = df[[
    'ppt_sum_min_1','tavg_mean_max_1','pdsi_mean_min_1','cwd_sum_max_1',
    'pet_mean_max_1', 'vpd_mean_max_1',
    'age', 'age_squared', 'duration'
]]
codes = df['type_code'].unique() 
type_code = df['type_code']

In [ ]:
inds = (np.isnan(x).sum(axis=1) == 0) & (~np.isnan(y)) & (y < 1)
y = y[inds].values
x = x[inds].values
type_code = type_code[inds]

In [ ]:
x_z, x_mean, x_std = utils.zscore_2d(x)

Fit models

In [ ]:
pf = pd.DataFrame()
pf['type_code'] = type_code
pf['observed'] = y
pf['predicted'] = np.NaN
pf['expected_value'] = np.NaN

codes = df['type_code'].unique() 
models = {}
for code in codes:
    fit_inds = type_code==code
    if (sum(y[fit_inds] > 0) > 10) & (sum(y[fit_inds] == 0) > 10):
        model = fit.hurdle(x=x_z[fit_inds], y=y[fit_inds])
        models[code] = model
        pf.loc[fit_inds, 'predicted'] = model.predict(x=x_z[fit_inds])
        pf.loc[fit_inds, 'expected_value'] = model.predict_expected_value(x=x_z[fit_inds])
        
pf = pf[pf['expected_value'] < 1]

In [ ]:
plt.plot(pf['observed'], pf['expected_value'],'.')

In [ ]:
pf[['observed', 'expected_value']].corr().iloc[0,1] ** 2

In [ ]:
plot.xy(
    x=pf['observed'], 
    y=pf['expected_value'], 
    xlim=(0,1), 
    ylim=(0,1), 
    xlabel='measured insect ratio', 
    ylabel='predicted insect ratio',
    width=300,
    height=300,
    opacity=0.1
)

Make a map of mortality ratios

In [ ]:
plot.carto(
    lat=df['lat'], 
    lon=df['lon'], 
    color=df['mort_ratio'],
    cmap='reds',
    clim=(0,0.3),
    size=10,
    opacity=0.5,
)

In [ ]:
plot.carto(
    lat=df['lat'], 
    lon=df['lon'], 
    color=pf['expected_value'],
    cmap='reds',
    clim=(0,0.05),
    size=5,
    opacity=0.75
)

Predict in the future

In [ ]:
df = load.fia(store='local', states='conus')
df = load.terraclim(
    store='local', 
    tlim=(2000,2010),
    data_vars=['ppt','tavg','pdsi','cwd','pet','vpd'], 
    data_aggs=['sum','mean','mean','sum','mean','mean'], 
    df=df, 
)

In [ ]:
df['age_squared'] = df['age'] ** 2
x = df[[
    'ppt_sum_min','tavg_mean_max','pdsi_mean_min','cwd_sum_max',
    'age', 'age_squared',
]]

In [ ]:
pf = pd.DataFrame()
pf['type_code'] = df['type_code']
pf['expected_value'] = np.NaN

codes = df['type_code'].unique() 
for code in codes:
    predict_inds = df['type_code']==code
    if (code in models.keys()):
        pf.loc[predict_inds, 'predicted'] = models[code].predict(x=x[predict_inds])
        pf.loc[predict_inds, 'expected_value'] = models[code].predict_expected_value(x=x[predict_inds])

In [ ]:
plot.carto(
    lat=df['lat'], 
    lon=df['lon'], 
    color=pf['expected_value'],
    cmap='reds',
    clim=(0,0.2),
    size=2,
)

In [ ]:
plot.carto(
    lat=df['lat'], 
    lon=df['lon'], 
    color=pf['expected_value'],
    cmap='reds',
    clim=(0,0.2),
    size=2,
)